In [21]:
import os
import re
from re import search
import xml.etree.ElementTree as ET
import sys
import numpy as np 

import nltk
from nltk.corpus import stopwords

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [22]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [23]:
files = os.listdir('xmlT/')

In [24]:
def xpathClean(file):
    XMLString = ""
    if os.path.isfile(os.path.join('xml/', file)):
        fr = open(os.path.join('xml/', file), "r", encoding="utf-8")
        #print (os.path.join('xml/', file))
        try:
            root = ET.parse(fr)
            result = '' 
            for element in root.findall('.//span'):
                if element.attrib.get('class'):
                    if "del" in element.attrib.get('class'):
                        #print(element.tag)
                        #print(element.attrib.get('data-time'))
                        #print(element.text)
                        element.tag = 'del'
                        #print(element.tag)
                        #print("")
                    elif "ins" in element.attrib.get('class'):
                        #print(element.tag)
                        #print(element.attrib.get('data-time'))
                        #print(element.text)
                        element.tag = 'ins'
                        #print(element.tag)
                        #print("")

            XML = root.getroot()
            XMLString = ET.tostring(XML, encoding='unicode')
        except:
            print("xpathClean - Oops!", sys.exc_info()[0], "occurred.")
            print (os.path.join('xml/', file))
        fr.close
       
    return XMLString

In [25]:
Words = []

for file in files:
    print(file)
    Changes = []
    try:
        line = xpathClean(file)
    except:
        print("Oops!", sys.exc_info()[0], "occurred.")
        
    #print(line)
    #print('in')
    
    line = re.sub(r' store="[^"]*"', '', line, flags=re.IGNORECASE)
    line = line.replace(' contenteditable="true"',"")
    line = re.sub(r'<journal>.*?<body1[^>]*>', '<body>', line, flags=re.IGNORECASE)
    line = re.sub(r'</body1>.*?</journal>', '</body>', line, flags=re.IGNORECASE)
    line = re.sub(r'<img[^>]+>','', line, flags=re.IGNORECASE)
    line = line.replace("</pp1>","</p>")
    line = line.replace("</title1>","</h3>")
    line = line.replace("<ins>−</ins>","−")
    line = line.replace("<del>-</del>","")
    line = line.replace("<del>percent</del>","")
    line = line.replace("<ins>%</ins>","%")
    line = line.replace("</del>","[/del]")
    line = line.replace("</ins>","[/ins]")
    line = re.sub(r'<pp1[^>]*>', '\n<p>', line, flags=re.IGNORECASE)
    line = re.sub(r'<title1 [^>]*>', '\n<h3>', line, flags=re.IGNORECASE)
    line = re.sub(r'<del[^>]*>', '[del]', line, flags=re.IGNORECASE)
    line = re.sub(r'<ins[^>]*>', '[ins]', line, flags=re.IGNORECASE)
    line = re.sub(r'<xref[^>]*>(.*?)</xref>', '\\1', line, flags=re.IGNORECASE)
    line = re.sub(r'<sec [^>]*>', '', line, flags=re.IGNORECASE)
    line = re.sub(r'<i [^>]*>', '', line, flags=re.IGNORECASE)
    line = re.sub(r'<query[^>]*>', '', line, flags=re.IGNORECASE)
    line = line.replace("</query>","")
    line = line.replace("</i>","")
    line = line.replace("</sec>","")
    line = re.sub(r'<label1[^>]*>(.*?)</label1>', '', line, flags=re.IGNORECASE)
    line = re.sub(r'<table-wrap[^>]*>(.*?)</table-wrap>', '', line, flags=re.IGNORECASE)

    line = re.sub("[<][^>]+[>]", "", line)
    line = re.sub("&lt;", "<", line)
    line = re.sub("&gt;", ">", line)
    line = re.sub("&amp;", "&", line)

    #Clear up for Sentance split
    line = re.sub(" e\.g\. ", " e[dot]g[dot] ", line)
    line = re.sub(" i\.e\. ", " i[dot]e[dot] ", line)
    line = re.sub("([0-9]+)\.([0-9]+)", "\\1[dot]\\2", line)

    #while search("(\[del[^\]]*\](.*?))\.((.*?)\[/del\])", line):
        #line = re.sub("(\[del[^\]]*\](.*?))\.((.*?)\[/del\])", "\\1[dot123]\\3", line)
    
    #for sent in re.split('[\.\?\!]', lineItem):
    #for sent in re.split('[\.|\?|\!]', lineItem):
    for l in line.split("\n"):
        #print('\n---\n' + s + '\n---\n')
        for s in l.split("."):
            #print("\n\n-----------\n" + s + "\n-------------\n")
            #print(str(search('\[del\]', sent)) + "---" + str(search('\[ins\]', sent)))
            if (search('\[del\]', s) or search('\[ins\]', s)):
                #print(s)
                if len(s) <= 100:
                    s = s.replace("[dot]",".")

                    orginalText = re.sub("\[del\]", "", s)
                    orginalText = re.sub("\[/del\]", "", orginalText)
                    orginalText = re.sub("\[ins[^\]]*\](.*?)\[/ins\]", "", orginalText)

                    correctedText = re.sub("\[ins\]", "", s)
                    correctedText = re.sub("\[/ins\]", "", correctedText)
                    correctedText = re.sub("\[del[^\]]*\](.*?)\[/del\]", "", correctedText)


                    Changes.append(orginalText.strip() +'\t'+ correctedText.strip() + '\t\n')
                    #print(orginalText +'\t'+ correctedText + '\t\n')

    fw = open(os.path.join('xml/collection/track.txt'), "a", encoding="utf-8")  

    fw.writelines(Changes)
    fw.close

ECC_190234_nlm_new_2019-08-14_150634.xml
ECC_190340_nlm_new_2019-12-04_141537.xml
ECC_190348_nlm_new_2019-10-15_105202.xml
ECC_190348_nlm_new_2019-10-15_121515.xml
ECC_190348_nlm_new_2019-10-15_160858.xml
